In [ ]:
import matplotlib.pyplot as plt
from plot_keras_history import plot_history

from study.hyper_study import ActivityRecognitionOptunaStudy
from util import vis

In [ ]:
hyperparameter_set_1 = {
    "segment_size": 80,
    "batch_size": 50,
    "optimizer": lambda trial: trial.suggest_categorical(
        "optimizer", ["adam", "sgd", "rmsprop", "adamax", "nadam"]
    ),
    "n_shifts": 5,
    "l_rate": lambda trial: trial.suggest_float("l_rate", 1e-4, 1e-2, log=True),
    "n_conv_layers": lambda trial: trial.suggest_int("n_conv_layers", 1, 3),
    "conv_filters": lambda trial, i: trial.suggest_int(
        f"conv_filters_{i}", 32, 128, step=32
    ),
    "recurrent_type": lambda trial: trial.suggest_categorical(
        "recurrent_type", ["GRU", "LSTM"]
    ),
    "n_recurrent_layers": lambda trial: trial.suggest_int("n_recurrent_layers", 1, 3),
    "recurrent_units": lambda trial, i: trial.suggest_int(
        f"recurrent_units_{i}", 32, 256, step=32
    ),
    "n_dense_layers": lambda trial: trial.suggest_int("n_dense_layers", 0, 1),
    "dense_units": lambda trial, i: trial.suggest_int(
        f"dense_units_{i}", 64, 256, step=64
    ),
    "dropout_rate": lambda trial: trial.suggest_float("dropout_rate", 0.2, 0.5),
    "use_batch_norm": lambda trial: trial.suggest_categorical(
        "use_batch_norm", [True, False]
    ),
    "pooling_type": lambda trial: trial.suggest_categorical(
        "pooling_type", ["max", "average"]
    ),
    "pool_after_each": lambda trial: trial.suggest_categorical(
        "pool_after_each", [True, False]
    ),
}

study = ActivityRecognitionOptunaStudy(
    epochs=1, hyperparameter_suggestions=hyperparameter_set_1
)

optuna_study = study.perform_trials(n_trials=2)

In [ ]:
images = vis.show_optuna_study_images(optuna_study)
images

In [ ]:
best_model = study.recreate_model_from_study(optuna_study)
vis.get_model_vis(best_model)

In [ ]:
history = best_model.train_model(max_epochs=1)
plt.show(plot_history(history))

In [ ]:
best_model.evaluate_model()